In [2]:
!ngrok config add-authtoken 2vGMWXrxn69OfjnRRq4urGRPXu1_XCvRa2nVXNtmtnas4gir


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
!pip install -q fastapi uvicorn sentence-transformers nest_asyncio pyngrok

from fastapi import FastAPI
from pydantic import BaseModel
from sentence_transformers import SentenceTransformer, util
from typing import List
import nest_asyncio
import uvicorn

# Required for Colab
nest_asyncio.apply()

app = FastAPI()
model = SentenceTransformer('all-MiniLM-L6-v2')

class QueryRequest(BaseModel):
    query: str
    questions: List[str]

@app.post("/similar")
def get_similar(data: QueryRequest):
    query_emb = model.encode(data.query, convert_to_tensor=True)
    question_embs = model.encode(data.questions, convert_to_tensor=True)
    scores = util.cos_sim(query_emb, question_embs)[0]
    top_results = sorted(zip(data.questions, scores), key=lambda x: x[1], reverse=True)[:5]
    return {"results": [{"question": q, "score": float(s)} for q, s in top_results]}


In [ ]:

# Tunnel via ngrok
from pyngrok import ngrok
public_url = ngrok.connect(8082)
print(f"Your public Colab API URL: {public_url}")

uvicorn.run(app, port=8082)


Your public Colab API URL: NgrokTunnel: "https://58c5-34-133-190-139.ngrok-free.app" -> "http://localhost:8082"


INFO:     Started server process [368]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8082 (Press CTRL+C to quit)


INFO:     203.145.49.160:0 - "POST /similar HTTP/1.1" 422 Unprocessable Entity
INFO:     203.145.49.160:0 - "POST /similar HTTP/1.1" 200 OK
INFO:     203.145.49.160:0 - "POST /similar HTTP/1.1" 200 OK
